In [1]:
#install required non-standard modules

!python3 -m pip install pandas > install_log.txt
!python3 -m pip install bokeh > install_log.txt
!python3 -m pip install midiutil > install_log.txt
!python3 -m pip install mido > install_log.txt
!python3 -m pip install pydub > install_log.txt

In [2]:
#import required modules

#for general purpose
import os
import re
import random
import datetime
import warnings
warnings.filterwarnings('ignore')

#for display and interactivity
from IPython.display import display, clear_output, Audio, Image
import ipywidgets as widgets

#for data and calculus
import pandas as pd
import numpy as np
from collections import Counter, defaultdict
import itertools, functools, operator

#for graphics
from bokeh.plotting import figure, show
from bokeh.io import show, output_notebook
from bokeh.models import ColumnDataSource, OpenURL, TapTool, Tabs, Panel

#for audio
from midiutil import MIDIFile
from mido import MidiFile
from pydub.generators import Sine
from pydub import AudioSegment

In [3]:
display(Image('logo_création(s).jpg', width = 100, height = 60))
output_notebook()
print('Install complete\n')
print('Setup complete\n\n')
print("Vous pouvez démarrer l'écoute de l'échantillon sonore en appuyant sur le bouton *Jouons...!*\n")
print("Le principe est plutôt simple:\n")
print("Écoutez attentivement...puis...\n")
print("Parmi les émotions proposées, cliquez l'histogramme de celles qui vous semblent 'appropriées' :-)")

FileNotFoundError: No such file or directory: 'logo_création(s).jpg'

FileNotFoundError: No such file or directory: 'logo_création(s).jpg'

<IPython.core.display.Image object>

Loading BokehJS ...

Install complete

Setup complete


Vous pouvez démarrer l'écoute de l'échantillon sonore en appuyant sur le bouton *Jouons...!*

Le principe est plutôt simple:

Écoutez attentivement...puis...

Parmi les émotions proposées, cliquez l'histogramme de celles qui vous semblent 'appropriées' :-)


In [4]:
#setup lists for transpose()
tone_flat = ['C','F','Bb','Eb','Ab','Db','Gb','Cb']
tone_sharp = ['G','D','A','E','B','F_','C_']
pitch_flat = ['Ab','A','Bb','Cb','C','Db','D','Eb','E','F','Gb','G']   
pitch_sharp = ['G_','A','A_','B','C','C_','D','D_','E','F','F_','G']

#setup dicts for voiceGen
C = {} ; F = {} ; Bb = {} ; Eb = {}; Ab = {} ; Db = {} ; Gb = {} ; Cb = {} ; 
G = {} ; D = {} ; A = {} ; E = {} ; B = {} ; F_ = {} ; C_ = {} ;

#the expressive values of the intervals according to Willems (1977)
#converted to a DataFrame

willemsDF = pd.read_csv('intervals_willems_1977_FR.csv', sep='\t')

#voicings DB in C according to chord_type
C['M7'] = [
    ['C3','B3','E4','G4'],
    ['C3','G3','B3','E4'],
    ['C3','G3','E4','B4'],
    ['C3','E3','B3','G4'],
    ['E3','B3','C4','G4'],
    ['E3','C4','G4','B4'],
    ['E3','G3','C4','B4'],
    ['G3','C4','E4','B4'],
    ['G3','B3','C4','E4'],
    ['C4','E4','G4','B4'],
]

C['6'] = [
    ['C3','A3','E4','G4'],
    ['C3','G3','A3','E4'],
    ['C3','G3','E4','A4'],
    ['C3','E3','A3','E4'],
    ['E3','A3','C4','G4'],
    ['E3','G3','C4','A4'],
    ['G3','A3','C4','E4'],
    ['E3','G3','A3','C4'],
    ['C4','E4','G4','A4'],
    ['E3','A3','D4','G4'],
]

C['m7b5'] = [
    ['C3','Bb3','Eb4','Gb4'],
    ['C3','Eb3','Bb3','Gb4'],
    ['C3','Gb3','Bb3','Eb4'],
    ['C3','Gb3','Eb4','Bb4'],
    ['C3','Eb4','Gb4','Bb4'],
    ['Eb3','Bb3','Gb4','C4'],
    ['Eb3','Bb3','C4','Gb4'],
    ['Eb3','Gb3','C4','Bb4'],
    ['Eb3','C4','Gb4','Bb4'],
    ['Gb3','C4','Eb4','Bb4'],
    ['Bb2','Gb3','Eb4','C5'],
    ['C4','Eb4','Gb4','Bb4'],
    ['Bb3','C4','Eb4','Gb4'],
    ['Gb3','Bb3','C4','Eb4'],
    ['Eb3','Gb3','Bb3','C4'],
    ['Gb3','C4','F4','Bb4'],
    ['C3','Gb3','Bb3','Eb4'],
]

C['dim7'] = [
    ['C3','A3','Eb4','Gb4'],
    ['C3','Gb3','Eb4','A4'],
    ['C3','Gb3','A3','Eb4'],
    ['C3','Eb3','A4','Gb4'],
    ['Eb3','C4','Gb4','A4'],
    ['Eb3','A4','C4','Gb4'],
    ['Eb3','A4','Gb4','C5'],
    ['Eb3','A4','C4','Gb4'],
    ['Gb3','C4','Eb4','A4'],
    ['Gb3','Eb4','A4','C5'],
    ['A2','Eb3','C4','Gb4'],
    ['A2','Eb3','Gb3','C4'],
    ['A4','Gb3','Eb4','C5'],
    ['A4','Gb3','C4','Eb4'],
    ['C4','Eb4','Gb4','A4'],
    ['A4','C4','Eb4','Gb4'],
    ['Gb3','A4','C4','Eb4'],
    ['Eb3','Gb3','A4','C4'],
]

C['mM7'] = [
    ['C3','B3','Eb4','G4'],
    ['C3','G3','B3','Eb4'],
    ['C3','G3','Eb4','B4'],
    ['Eb3','B3','C4','G4'],
    ['Eb3','G3','C4','B4'],
    ['Eb3','C4','G4','B4'],
    ['G3','C4','Eb4','B4'],
    ['C4','Eb4','G4','B4'],
    ['B3','C4','Eb4','G4'],
    ['G3','B3','C4','Eb4'],
]

C['m7'] = [
    ['C3','Bb3','Eb4','G4'],
    ['C3','G3','Bb3','Eb4'],
    ['C3','G3','Eb4','Bb4'],
    ['C3','Eb3','Bb3','G4'],
    ['Eb3','Bb3','G4','C5'],
    ['Eb3','Bb3','C4','Bb3'],
    ['Eb3','G3','C4','Bb4'],
    ['Eb3','C4','G4','Bb4'],
    ['G3','C4','Eb4','Bb4'],
    ['Bb3','Eb4','G4','C5'],
    ['Bb2','G3','Eb4','C5'],
    ['C4','Eb4','G4','Bb4'],
    ['Bb3','C4','Eb4','G4'],
    ['G3','Bb3','C4','Eb4'],
    ['Eb3','G3','Bb3','C4'],
]

C['m6'] = [
    ['C3','A3','Eb4','G4'],
    ['C3','G3','A3','Eb4'],
    ['C3','G3','Eb4','A4'],
    ['C3','Eb3','A3','G4'],
    ['Eb3','C4','G4','A4'],
    ['G3','C4','Eb4','A4'],
    ['A2','G3','C4','Eb4'],
    ['C4','Eb4','G4','A4'],
    ['A3','C4','Eb4','G4'],
    ['G3','A3','C4','Eb4'],
    ['Eb3','G3','A3','C4'],
    ['Eb3','A3','D4','G4'],
]

C['7'] = [
    ['C3','Bb3','E4','G4'],
    ['C3','G3','Bb3','E4'],
    ['C3','G3','E4','Bb4'],
    ['C3','E3','Bb3','G4'],
    ['E3','Bb3','G4','C5'],
    ['E3','Bb3','C4','G4'],
    ['E3','G3','C4','Bb4'],
    ['E3','C4','G4','Bb4'],
    ['G3','C4','E4','Bb4'],
    ['Bb3','C4','E4','C5'],
    ['Bb2','G3','E4','C5'],
    ['Bb2','E3','A3','D4'],
]

#function that take a chord in 'input' and transpose it from 'initkey' to 'finalkey'

def transpose(chord,initkey,finalkey):
    
    note_to_midi_flat = {
    'A0':'21','Bb0':'22','B0':'23','C1':'24','Db1':'25','D1':'26','Eb1':'27','E1':'28','F1':'29','Gb1':'30','G1':'31','Ab1':'32','A1':'33','Bb1':'34','B1':'35','C2':'36','Db2':'37','D2':'38','Eb2':'39','E2':'40','F2':'41','Gb2':'42','G2':'43','Ab2':'44','A2':'45','Bb2':'46','B2':'47','C3':'48','Db3':'49','D3':'50','Eb3':'51','E3':'52','F3':'53','Gb3':'54','G3':'55','Ab3':'56','A3':'57','Bb3':'58','B3':'59','C4':'60','Db4':'61','D4':'62','Eb4':'63','E4':'64','F4':'65','Gb4':'66','G4':'67','Ab4':'68','A4':'69','Bb4':'70','B4':'71','C5':'72','Db5':'73','D5':'74','Eb5':'75','E5':'76','F5':'77','Gb5':'78','G5':'79','Ab5':'80','A5':'81','Bb5':'82','B5':'83','C6':'84','Db6':'85','D6':'86','Eb6':'87','E6':'88','F6':'89','Gb6':'90','G6':'91','Ab6':'92','A6':'93','Bb6':'94','B6':'95','C7':'96','Db7':'97','D7':'98','Eb7':'99','E7':'100','F7':'101','Gb7':'102','G7':'103','Ab7':'104','A7':'105','Bb7':'106','B7':'107','C8':'108',
    }
    midi_to_note_flat = {v:k for k,v in note_to_midi_flat.items()}
    
    note_to_midi_sharp = {
    'A0':'21','A#0':'22','B0':'23','C1':'24','C#1':'25','D1':'26','D#1':'27','E1':'28','F1':'29','F#1':'30','G1':'31','G#1':'32','A1':'33','A#1':'34','B1':'35','C2':'36','C#2':'37','D2':'38','D#2':'39','E2':'40','F2':'41','F#2':'42','G2':'43','G#2':'44','A2':'45','A#2':'46','B2':'47','C3':'48','C#3':'49','D3':'50','D#3':'51','E3':'52','F3':'53','F#3':'54','G3':'55','G#3':'56','A3':'57','A#3':'58','B3':'59','C4':'60','C#4':'61','D4':'62','D#4':'63','E4':'64','F4':'65','F#4':'66','G4':'67','G#4':'68','A4':'69','A#4':'70','B4':'71','C5':'72','C#5':'73','D5':'74','D#5':'75','E5':'76','F5':'77','F#5':'78','G5':'79','G#5':'80','A5':'81','A#5':'82','B5':'83','C6':'84','C#6':'85','D6':'86','D#6':'87','E6':'88','F6':'89','F#6':'90','G6':'91','G#6':'92','A6':'93','A#6':'94','B6':'95','C7':'96','C#7':'97','D7':'98','D#7':'99','E7':'100','F7':'101','F#7':'102','G7':'103','G#7':'104','A7':'105','A#7':'106','B7':'107','C8':'108',
    }
    midi_to_note_sharp = {v:k for k,v in note_to_midi_sharp.items()}
    
    if initkey in tone_flat:
        init = pitch_flat.index(initkey)
    else: 
        init = pitch_sharp.index(initkey)    
    
    if finalkey in tone_flat:
        final = pitch_flat.index(finalkey)
    else:
        final = pitch_sharp.index(finalkey)

    diff = final - init
    
    try:
        if finalkey in tone_flat:
            chord_index = note_to_midi_flat[chord]
        else:
            chord_index = note_to_midi_sharp[chord]
    except KeyError:
        chord_index = note_to_midi_flat[chord]


    chord_transposed_index = int(chord_index) + diff
    
    if finalkey in tone_flat:
        chord_transposed = midi_to_note_flat[str(chord_transposed_index)]
    else:
        chord_transposed = midi_to_note_sharp[str(chord_transposed_index)]

    return chord_transposed

#chord_type list used by Hume keyGen routine below
chord_type = ['6','7','M7','dim7','m6','m7','m7b5','mM7']

#generate all Hume chords by calculating cartesian product of tone (ex: C) and chord_type (ex: m7)
#flat and sharp tonality are treated separately for enharmonic issues
#chords dicts are executed (exec) to build them
k = ''
for i,j in itertools.product(tone_flat,chord_type):
    k = i
    i = {j:''}
    i[j] = [[transpose(x,'C',k) for x in chord] for chord in C[j]]
    exec(k+'[\''+j+'\']'+'='+str(i[j]))
    
for i,j in itertools.product(tone_sharp,chord_type):
    k = i
    i = {j:''}
    i[j] = [[transpose(x,'C',k) for x in chord] for chord in C[j]]
    exec(k+'[\''+j+'\']'+'='+str(i[j]))

#audioGen module n°1
#convert note to freq for wav gen
def note_to_freq(note, concert_A=440.0):
    '''
    from wikipedia: http://en.wikipedia.org/wiki/MIDI_Tuning_Standard#Frequency_values
    '''
    return (2.0 ** ((note - 69) / 12.0)) * concert_A

#audioGen module n°2
#convert midi ticks to ms for wav gen
def ticks_to_ms(ticks,mid_file):
    tick_ms = (60000.0 / tempo) / mid_file.ticks_per_beat
    return ticks * tick_ms

#audioGen
#midi constant declaration
track001 = 0
channel  = 0
time = 0    # In beats
duration001 = 4    # In beats
tempo = 100   # In BPM
volume = 100  # 0-127, as per the MIDI standard

#audioGen
#corresponsdance table note/midi
note_to_midi = {
    'A0':'21','A#0':'22','A1':'33','A#1':'34','A2':'45','A#2':'46','A3':'57','A#3':'58','A4':'69','A#4':'70','A5':'81','A#5':'82','A6':'93','A#6':'94','A7':'105','A#7':'106','Ab1':'32','Ab2':'44','Ab3':'56','Ab4':'68','Ab5':'80','Ab6':'92','Ab7':'104','B0':'23','B1':'35','B2':'47','B3':'59','B4':'71','B5':'83','B6':'95','B7':'107','Bb0':'22','Bb1':'34','Bb2':'46','Bb3':'58','Bb4':'70','Bb5':'82','Bb6':'94','Bb7':'106','C1':'24','C#1':'25','C2':'36','C#2':'37','C3':'48','C#3':'49','C4':'60','C#4':'61','C5':'72','C#5':'73','C6':'84','C#6':'85','C7':'96','C#7':'97','C8':'108','D1':'26','D#1':'27','D2':'38','D#2':'39','D3':'50','D#3':'51','D4':'62','D#4':'63','D5':'74','D#5':'75','D6':'86','D#6':'87','D7':'98','D#7':'99','Db1':'25','Db2':'37','Db3':'49','Db4':'61','Db5':'73','Db6':'85','Db7':'97','E1':'28','E2':'40','E3':'52','E4':'64','E5':'76','E6':'88','E7':'100','Eb1':'27','Eb2':'39','Eb3':'51','Eb4':'63','Eb5':'75','Eb6':'87','Eb7':'99','F1':'29','F#1':'30','F2':'41','F#2':'42','F3':'53','F#3':'54','F4':'65','F#4':'66','F5':'77','F#5':'78','F6':'89','F#6':'90','F7':'101','F#7':'102','G1':'31','G#1':'32','G2':'43','G#2':'44','G3':'55','G#3':'56','G4':'67','G#4':'68','G5':'79','G#5':'80','G6':'91','G#6':'92','G7':'103','G#7':'104','Gb1':'30','Gb2':'42','Gb3':'54','Gb4':'66','Gb5':'78','Gb6':'90','Gb7':'102'
}

def audioGen(query):
    #########
    #audioGen
    #########

    #
    # midi gen
    #
    #create midi objects
    MyMIDI = MIDIFile(1)
    MyMIDI.addTempo(track001, time, tempo)
    
    #get chord progression from query
    
    query = re.sub('#','_',query)
    query = re.findall(r'(C|F|Bb|Eb|Ab|Db|Gb|Cb|G|D|A|E|B|F_|C_)(6|7|M7|dim7|m6|m7\b|m7b5|mM7)',query)
    
    chords = [eval(i[0]+"['"+i[1]+"']") for i in query]
    chord_progressions = list(itertools.product(*chords))
    
    j = random.choice(chord_progressions)

#     j = eval(j)
    
    #formatting chords list for filename handling 
    k = str(j)
    k = k.replace(' ','')
    k = k.replace('(','')
    k = k.replace(')','')
    k = k.replace('[','')
    k = k.replace(']','')
    k = k.replace(',','')
    k = k.replace('\'','')
    k = k.replace('#','_')
    
    #counter for separating chords in list
    x = 0

    for z in range(len(j)):
        for i in range(0,4):
            MyMIDI.addNote(track001,channel,int(note_to_midi[j[z][i]]),time+x,duration001,volume)
        x += 4

    #write midi file
    output_file = open('_'+k+'.mid', 'wb')
    MyMIDI.writeFile(output_file)
    output_file.close()

    #
    # wav gen
    #

    mid = MidiFile('_'+k+'.mid')

    output = AudioSegment.silent(mid.length * 1000.0)

    for track in mid.tracks:
        # position of rendering in ms
        current_pos = 0.0

        current_notes = defaultdict(dict)
        # current_notes = {
        #   channel: {
        #     note: (start_time, message)
        #   }
        # }

        for msg in track:
            current_pos += ticks_to_ms(msg.time,mid)

            if msg.type == 'note_on':
                current_notes[msg.channel][msg.note] = (current_pos, msg)

            if msg.type == 'note_off':
                start_pos, start_msg = current_notes[msg.channel].pop(msg.note)

                duration = current_pos - start_pos

                signal_generator = Sine(note_to_freq(msg.note))
                rendered = signal_generator.to_audio_segment(duration=duration-50, volume=-20).fade_out(100).fade_in(30)

                output = output.overlay(rendered, start_pos)

    output.export('_'+k+'.wav', format="wav")
    os.remove('_'+k+'.mid')

    audioGen_path = '_'+k+'.wav'

    del output_file
    del mid
    del MyMIDI
    
    return display(Audio(filename='_'+k+'.wav', autoplay=True))


#function that take a progression 'query' (ex: Cm7 F7) in input, convert it interv

def convert_to_intervals(query):
    query_1 = re.sub('#','_',query)
    query_1 = re.findall(r'(C|F|Bb|Eb|Ab|Db|Gb|Cb|G|D|A|E|B|F_|C_)(6|7|M7|dim7|m6|m7\b|m7b5|mM7)',query_1)
    query_1 = np.array(query_1)
    chords = np.array([eval(i[0]+"['"+i[1]+"']") for i in query_1], dtype=object)
    
    len_chord_progressions = functools.reduce(operator.mul, map(len, chords), 1)
    
    #corresponsdance table note/midi
    note_to_midi = {
        'A0':'21','A#0':'22','A1':'33','A#1':'34','A2':'45','A#2':'46','A3':'57','A#3':'58','A4':'69','A#4':'70','A5':'81','A#5':'82','A6':'93','A#6':'94','A7':'105','A#7':'106','Ab1':'32','Ab2':'44','Ab3':'56','Ab4':'68','Ab5':'80','Ab6':'92','Ab7':'104','B0':'23','B1':'35','B2':'47','B3':'59','B4':'71','B5':'83','B6':'95','B7':'107','Bb0':'22','Bb1':'34','Bb2':'46','Bb3':'58','Bb4':'70','Bb5':'82','Bb6':'94','Bb7':'106','C1':'24','C#1':'25','C2':'36','C#2':'37','C3':'48','C#3':'49','C4':'60','C#4':'61','C5':'72','C#5':'73','C6':'84','C#6':'85','C7':'96','C#7':'97','C8':'108','D1':'26','D#1':'27','D2':'38','D#2':'39','D3':'50','D#3':'51','D4':'62','D#4':'63','D5':'74','D#5':'75','D6':'86','D#6':'87','D7':'98','D#7':'99','Db1':'25','Db2':'37','Db3':'49','Db4':'61','Db5':'73','Db6':'85','Db7':'97','E1':'28','E2':'40','E3':'52','E4':'64','E5':'76','E6':'88','E7':'100','Eb1':'27','Eb2':'39','Eb3':'51','Eb4':'63','Eb5':'75','Eb6':'87','Eb7':'99','F1':'29','F#1':'30','F2':'41','F#2':'42','F3':'53','F#3':'54','F4':'65','F#4':'66','F5':'77','F#5':'78','F6':'89','F#6':'90','F7':'101','F#7':'102','G1':'31','G#1':'32','G2':'43','G#2':'44','G3':'55','G#3':'56','G4':'67','G#4':'68','G5':'79','G#5':'80','G6':'91','G#6':'92','G7':'103','G#7':'104','Gb1':'30','Gb2':'42','Gb3':'54','Gb4':'66','Gb5':'78','Gb6':'90','Gb7':'102'
        }
    
    chord_mid = np.array([[[k for k in range(4)] for j in range(len(query_1))] for x in range(len_chord_progressions)])
    chord_intervals = [[j for j in range(len(query_1))] for x in range(len_chord_progressions)]
    chord_intervals_abs = [[j for j in range(len(query_1))] for x in range(len_chord_progressions)]
    
    for n,chord_progressions in enumerate(itertools.product(*chords)):
        for j in range(len(query_1)):
            for k in range(4):
                chord_mid[n][j][k] = note_to_midi[chord_progressions[j][k]]
    
    for n1,prog_mid in enumerate(chord_mid):
        for n2 in range(len(prog_mid)):
            chord_intervals[n1][n2] = list((itertools.combinations(prog_mid[n2],2)))
    
    chord_intervals_abs = [[[abs(interval[0]-interval[1])%12 for interval in intervals] for intervals in progressions] for progressions in chord_intervals]
    chord_intervals_abs = random.choice(chord_intervals_abs)
    query = query.replace('#','_')
    query_intervals = [query,chord_intervals_abs]
    
    return query_intervals

def display_emotions(query_intervals):
    
    categories = ['Sensoriel','Affectif','Intellectuel']
    tabs = Tabs()
    ltabs = []
    
    for cat in categories:
        interval_to_emotion = { willemsDF.loc[i,'Intervalle']:random.choice(str(willemsDF.loc[i,cat]).split(',')) for i in willemsDF.index }
        c = Counter()
        df = pd.DataFrame([[interval_to_emotion[interval] for interval in chord] for chord in query_intervals[1]])
        for n,x in enumerate(df.values):
            c += Counter(df.loc[n])
        c = c.most_common()
        c = dict(c)
        
        df_plot = pd.DataFrame.from_dict(c, orient='index', columns=['occurence'])
        df_plot['chords'] = str(query_intervals[0])
        
        source = ColumnDataSource(df_plot)
        
        p = figure(plot_width=400, plot_height=600, y_range=list(df_plot.index.unique()), x_range=(0, df_plot['occurence'].max() * 1.1), title=cat)
        p.hbar(y='index', left=0, right='occurence', height=0.2, source=source)
        p.ygrid.grid_line_color = None
        p.xaxis.axis_label = 'Occurence'
        p.outline_line_color = None
        
        tab = Panel(child=p, title=cat)
        ltabs.append(tab)
        
        url = "https://www.jazzreal.org/dome/data_log?emotion=@index&chords=@chords"
        taptool = p.tools.append(TapTool(callback=OpenURL(url=url)))


    tabs = Tabs(tabs=ltabs)
    show(tabs)
    

def select_random_progression():
    
    progression = ''
    root_list = ['F','Bb','Eb','Ab','Db','Gb','Cb','G','D','A','E','B','F#','C#']
    chord_type_list = ['6','7','M7','dim7','m6','m7','m7b5','mM7']
    
    for n in range(4):
        progression += random.choice(root_list)
        progression += random.choice(chord_type_list)
        progression += ' '
    
    print(progression)
    
    return progression    

def on_button_clicked(_):
    # "linking function with output"
    with out:
        # what happens when we press the button
        clear_output()
        random_progression = select_random_progression()
        audioGen(random_progression)
        display_emotions(convert_to_intervals(random_progression))
        print("On recommence ?!")

button = widgets.Button(description='Jouons...!')
out = widgets.Output()

# linking button and function together using a button's method
button.on_click(on_button_clicked)
# displaying button and its output together
widgets.VBox([button,out])

FileNotFoundError: [Errno 2] No such file or directory: 'intervals_willems_1977_FR.csv'